> `Text preprocessing script`
> ===


Importing packages for it


In [2]:
import string
import os
import json

>## Etapes que nous suivrons 
>- Choisir une langue pour la quelle nous souhaitons netoyer les données
>- Charger les ID de chapitres et livres de cette langue
>- parcourir les dossiers des livres et ouvrir les dossiers de chaque livre et les preprocess
>-  ...

Defining constants of the script


In [3]:
LANG = 63
# root folder for not processed data
DATA_IN_PATH = os.path.join(os.getcwd(), f'data/{LANG}')

# root folder for processed data
DATA_OUT_PATH = os.path.join(os.getcwd(), f'preprocessed_data/{LANG}')
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

# Sessions path
PREPROCESSING_SESSIONS_PATH = os.path.join(os.getcwd(),'preprocessing_sessions.json')
if not os.path.isfile(PREPROCESSING_SESSIONS_PATH):
    with open(PREPROCESSING_SESSIONS_PATH, 'w', encoding='utf-8') as sessions_file:
        sessions_file.write('{}')

SCRAPING_SESSIONS_PATH = os.path.join(os.getcwd(),'sessions.json')
CHAPTERS_FILE_PATH = os.path.join(os.getcwd(), f'utils/chapitres_id/{LANG}.json')



Defining functions tools

In [4]:

# dossier dans lequel on va mettre les dossiers chaque chapitre pour cette langue
def get_lang_folder_path(lang: int = LANG) -> str:
    path = os.path.join(os.getcwd(), f"preprocessed_data/{lang}")
    if not os.path.exists(path):
        os.makedirs(path)
    return path


# dossier dans lequel on va mettre les fichiers txt pour chaqe chap du livre livre
def get_book_folder_path_for_lang(book: str, lang: int = LANG) -> str:
    path = os.path.join(os.getcwd(), f"preprocessed_data/{lang}/{book}").replace('\\', '/')
    if not os.path.exists(path):
        print(f'Path to saves folder created : {path} ')
        os.makedirs(path)
    return path


def update_json_file(data,  file_path: str, type: type = list, lang: int = LANG):
    path = os.path.join(os.getcwd(), file_path)
    # initializing the file if not exists
    if not os.path.exists(path):
        with open(path, 'w', encoding='utf-8') as file:
            file.write('[]') if type == list else file.write('{}')
    # getting the data in the file and updating it
    with open(path, 'r+', encoding='utf-8') as file:
        file_data = json.load(file)
        if type == list:  # si nous sommes dans un liste
            file_data.append(data)
        else:  # si c'est un dictionnaire plutot
            file_data[f'{lang}'] = data
    # write the final data in the file
    with open(path, 'w', encoding='utf-8') as file:     
        json.dump(file_data, file, indent=4)

In [5]:
def save_current_session(book: str, chapter: str, completed_books: list[str], path=PREPROCESSING_SESSIONS_PATH, lang: int = LANG):

    session = {
        "last_book": book,
        "last_chapter": chapter,
        "completed_books": completed_books
    }
    update_json_file(session, path, dict)


def load_last_session(lang: int = LANG, path: str = PREPROCESSING_SESSIONS_PATH):
    default_session = {
        "last_book": '',
        "last_chapter": '',
        "completed_books": []
    }
    if not os.path.exists(path):
        with open(path, 'w', encoding='utf-8') as sessions_file:
            json.dump({lang: default_session}, sessions_file, indent=4)
        return default_session

    with open(path, 'r', encoding='utf-8') as sessions_file:
        sessions = json.load(sessions_file)
        try:
            return sessions[f'{lang}']
        except KeyError:
            return default_session


def load_scrapped_books(lang: str = LANG):
    last_scraping_session = load_last_session(path=SCRAPING_SESSIONS_PATH)
    books = last_scraping_session['completed_books']
    if len(books) == 0:
        print("Aucun livre à priorit disponible pour cette langue, veuillez lancer le script de scraping pour cela avant de revenir au preprocessing")
        exit()
    return books



def load_all_chapters(lang:str=LANG):
    with open(CHAPTERS_FILE_PATH, 'r', encoding='utf-8') as chapters_file:
        lang_chapters = json.load(chapters_file)
        
    return lang_chapters
    
def json_print(message, json_doc):
    print(message)
    print(json.dumps(json_doc, indent=4, separators=(',', ': ')))


In [7]:
# loading all scrapped books, if there is no scrapped book, we will exit the application
scraped_books = load_scrapped_books()
# Loading all the chapters for each existing books of the lang
books_chapters = load_all_chapters()
# Loading the last preprocessing session
last_session = load_last_session()
# Defining the list of the not preproceeded books but scraped
books_to_preprocess = list(set(scraped_books) - set(last_session['completed_books']))

have_to_complete_a_book = last_session['last_book'] != '' and last_session['last_book'] not in last_session['completed_books']

#  Ensure starting the session with the last books (if not completed) in preprocessing
if have_to_complete_a_book:
    last_book = last_session['last_book']
    books_to_preprocess.remove(last_book)
    books_to_preprocess = [last_book]+books_to_preprocess

# current chapter



Now we can start 